# Log data

## load libraries

In [1]:
import os.path

%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import time

import pandas as pd

import dateutil.parser

from io import StringIO

import datetime

import requests
import ssl

In [2]:
#base_url='http://nautilusdataviz.media.mit.edu:8888/files/'
#base_url='https://external.oet.org/science/monitor/'
base_url='http://shiphouse.nautilus/science/monitor/'
filenames=['Herc-Depth.samples','Temperature.samples','Salinity.samples']

In [3]:
colnames=['Salinity','Temperature']

# Reset log file

In [4]:
for colname in colnames:
    logfile='log_'+colname+'.samples'
    if (os.path.isfile(logfile)): 
        os.remove(logfile)

# support functions

In [5]:
def getDataVar(colname,base_url):
    
    filename=colname+'.samples'
    #response = requests.urlopen(base_url+filename,context=ctx)
    #data = response.read()      # a `bytes` object
    #text = data.decode('utf-8') 
    
    r = requests.get(base_url+filename,verify=False)
    text=r.text
    TESTDATA=StringIO(text)
    dat = pd.read_csv(TESTDATA, sep=" ",names=['date',colname])
    #dat = convert_unix_time(dat)
    return dat

In [6]:
#convert to unix time

def convert_unix_time(df):
    col = 'date'
    for row in range(0,len(df.index)):
        date=df[col].tolist()[row]
        pdate=dateutil.parser.parse(date)
        unixtime=int(time.mktime(pdate.timetuple()))
        df.set_value(row,col,unixtime)
    return df

In [7]:

def log(colname,base_url):

    filename=colname+'.samples'

    logfile='log_'+filename
    
   #print(filename,logfile)
    
    # get the latest data
    dat= getDataVar(colname,base_url)
    
    # convert the time
    
    dat = convert_unix_time(dat)
    
    
    if (os.path.isfile(logfile) == False):  # then log file
        dat.to_csv(logfile, sep=' ', header=False, index=False)

        
    else: 
        # get the log file
        log=pd.read_csv(logfile,delimiter=' ',names=['date','value'])

        # get the last log datapoint
        log_last=log['date'].iloc[-1]
        #print(log_last)

        # get the last dat datapoint
        dat_last=dat['date'].iloc[-1]
        #print(dat_last)

        if (log_last != dat_last):

            # then we don't have this data, add this last line to the file
            dat.tail(1).to_csv(logfile, sep=' ', mode='a', header=False, index=False)
            print("logging "+logfile)

            # improve this by figuring out how many data points are not in common ...

    log=pd.read_csv(logfile,delimiter=' ',names=['date','value'])


In [8]:


def group_log(colnames):

    for colname in colnames:
        log(colname,base_url)
        


In [9]:
while True:
    
    group_log(colnames)
    output="logged "
    ts = time.time()
    st = datetime.datetime.fromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S')
    print(output+st)
        
    time.sleep(30)

logged 2016-07-24 22:10:11
logged 2016-07-24 22:10:41
logging log_Salinity.samples
logging log_Temperature.samples
logged 2016-07-24 22:11:12
logged 2016-07-24 22:11:42
logging log_Salinity.samples
logged 2016-07-24 22:12:12
logged 2016-07-24 22:12:42
logging log_Salinity.samples
logging log_Temperature.samples
logged 2016-07-24 22:13:12
logged 2016-07-24 22:13:42
logged 2016-07-24 22:14:13
logged 2016-07-24 22:14:43
logging log_Salinity.samples
logging log_Temperature.samples
logged 2016-07-24 22:15:13
logged 2016-07-24 22:15:43
logged 2016-07-24 22:16:13
logged 2016-07-24 22:16:43
logging log_Salinity.samples
logging log_Temperature.samples
logged 2016-07-24 22:17:13
logged 2016-07-24 22:17:44
logged 2016-07-24 22:18:14
logged 2016-07-24 22:18:44
logging log_Salinity.samples
logging log_Temperature.samples
logged 2016-07-24 22:19:14
logged 2016-07-24 22:19:44
logging log_Salinity.samples
logging log_Temperature.samples
logged 2016-07-24 22:20:14
logged 2016-07-24 22:20:44
logged 2016

ConnectionError: HTTPConnectionPool(host='shiphouse.nautilus', port=80): Max retries exceeded with url: /science/monitor/Temperature.samples (Caused by <class 'OSError'>: [Errno 101] Network is unreachable)